<a href="https://colab.research.google.com/github/hsallrounder/Neural-Networks-Workshop-2023/blob/main/Neural_Networks_Lab_07_04_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Heart Disease Prediction Using Back Propagation

In [82]:
import pandas as pd
import numpy as np

In [83]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/heart.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [84]:
X = df.drop('target', axis = 1)
Y = df['target']

In [85]:
from sklearn.preprocessing import StandardScaler
mms = StandardScaler()
X = pd.DataFrame(mms.fit_transform(X),columns=["age","sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope","ca","thal"])

In [86]:
X.head()
Y.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

In [87]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_sizes, output_size):
        sizes = [input_size] + hidden_sizes + [output_size]
        self.weights = [np.random.randn(sizes[i], sizes[i+1]) for i in range(len(sizes)-1)]

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def feed_forward(self, X):
        self.layers = [X]
        for i in range(len(self.weights)):
            layer = self.sigmoid(np.dot(self.layers[-1], self.weights[i]))
            self.layers.append(layer)
        return self.layers[-1]

    def back_propagation(self, X, y, learning_rate):
        output_error = y - self.layers[-1]
        output_delta = output_error * self.sigmoid_derivative(self.layers[-1])

        deltas = [output_delta]
        for i in range(len(self.weights)-1, 0, -1):
            hidden_error = np.dot(deltas[-1], self.weights[i].T)
            hidden_delta = hidden_error * self.sigmoid_derivative(self.layers[i])
            deltas.append(hidden_delta)
        deltas.reverse()

        for i in range(len(self.weights)):
            self.weights[i] += learning_rate * np.dot(self.layers[i].T, deltas[i])

    def train(self, X, y, learning_rate, epochs):
        # print('Initial Weights',self.weights)
        for i in range(epochs):
            self.feed_forward(X)
            self.back_propagation(X, y, learning_rate)
        # print('Final Weights',self.weights)

    def predict(self, X):
        result = np.round(self.feed_forward(X)).astype(int)
        return result

In [88]:
print(X.shape)
print(Y.shape)

(1025, 13)
(1025,)


In [89]:
X_arr = np.array(X)
y_arr = np.array(Y).reshape(-1, 1)

# Create a neural network with X.shape[1] input nodes, 1 hidden layer of X.shape[1]/2 nodes , and 1 output node
nn = NeuralNetwork(X.shape[1], [int(X.shape[1]/2)], 1)

# Train the neural network for l_r=0.5 and 20 epochs
nn.train(X_arr, y_arr, 0.1, 20)

# Make predictions on the input data
predictions = nn.predict(X)
y_pred = pd.DataFrame(predictions)
y_pred = y_pred[0]

In [90]:
y_pred

0       0
1       0
2       0
3       0
4       0
       ..
1020    1
1021    0
1022    0
1023    1
1024    0
Name: 0, Length: 1025, dtype: int64

In [91]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_arr, y_pred))

0.9063414634146342
